In [2]:
# 1. Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

In [3]:
# Load environment variables
load_dotenv()

# PostgreSQL connection setup
DB_USER = os.getenv("POSTGRES_USER", "postgres")
DB_PASS = os.getenv("POSTGRES_PASSWORD", "postgres")
DB_HOST = os.getenv("POSTGRES_HOST", "db")
DB_PORT = os.getenv("POSTGRES_PORT", "5432")
DB_NAME = os.getenv("POSTGRES_DB", "telegram_data_db")

DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)

In [17]:
query = """
SELECT * FROM public_staging.stg_telegram_messages;
"""
df = pd.read_sql(query, engine)
df

,id,source_file,date,text,views,from_id,replies,forwards
0,1,CheMed123_2025-07-17,2023-02-10 12:23:06,"⚠️Notice!\nDear esteemed customers,\nDue to fo...",1222.0,NaN,NaN,1.0
1,2,CheMed123_2025-07-17,2023-02-02 08:58:52,Mela-One በውስጡ ሆርሞን ያለው ድንገተኛ ወሊድ መቆጣጠርያ ሲሆን ያለ...,1123.0,NaN,NaN,3.0
2,3,CheMed123_2025-07-17,2023-02-01 08:59:37,አዚትሮማይሲን በሃኪም መድሃኒት ማዘዣ ከሚታዘዙ አንቲባዮቲኮች አንዱ ሲሆን...,1015.0,NaN,NaN,4.0
3,4,CheMed123_2025-07-17,2023-01-31 09:19:53,Che-Med Trivia #3\n\nምግብና መጠጦች አንዳንድ መድሃኒቶች በደ...,784.0,NaN,NaN,1.0
4,5,CheMed123_2025-07-17,2023-01-30 09:45:25,"Che-Med Trivia #2\n\nእንደ Ciprofloxacin, Doxycy...",681.0,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...
3374,3375,tikvahpharma_2025-07-17,2025-07-14 17:00:32,PHARMAKON TRADING HOUSE PLC(IMPORTER)\n\n ♟G...,775.0,1.353258e+09,0.0,0.0
3375,3376,tikvahpharma_2025-07-17,2025-07-14 16:28:24,Tikvah Sales\n(Pharma Import)\n\n⬇️🆕arrival \n...,997.0,1.353258e+09,0.0,1.0
3376,3377,tikvahpharma_2025-07-17,2025-07-14 15:18:57,NEW JOB VACCANCY \n08/11/2017\nJUNIOR PHARMACY...,716.0,1.353258e+09,0.0,3.0
3377,3378,tikvahpharma_2025-07-17,2025-07-14 14:59:12,From import \n\n🚨 New Arrival: Medical Equipm...,719.0,1.353258e+09,0.0,0.0


In [16]:
# 1. Frequent product mentions (simplified)

query = """
SELECT
  word,
  COUNT(*) as mention_count
FROM (
  SELECT
    unnest(string_to_array(lower(text), ' ')) as word
  FROM public_staging.stg_telegram_messages
) sub
WHERE word NOT IN ('', '-', 'of', 'and', 'the', 'a', 'an', 'in', 'on', 'for', 'to', 'with', 'is', 'are', 'was', 'were', 'by', 'at', 'as', 'that', 'this', 'from', 'እስከ', 'until', 'high', 'ከሰኞ')
GROUP BY word
ORDER BY mention_count DESC
LIMIT 20;
"""
df = pd.read_sql(query, engine)
df

,word,mention_count
0,birr,4006
1,monday,2631
2,pharmacy,1548
3,bole,1523
4,delivery,1521
...,...,...
195,pulse,145
196,baby,143
197,monitor,142
198,🗳,142


In [15]:
# 3. Posting patterns (daily)
query = """
SELECT
  DATE_TRUNC('day', date) AS day,
  COUNT(*) AS message_count
FROM public_staging.stg_telegram_messages
GROUP BY day
ORDER BY 2 desc;
"""
df = pd.read_sql(query, engine)
df

,day,message_count
0,2025-07-17,126
1,2025-07-16,174
2,2025-07-15,153
3,2025-07-14,111
4,2025-07-13,42
...,...,...
87,2022-09-16,3
88,2022-09-10,3
89,2022-09-07,3
90,2022-09-06,3
